In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "false")
print("spark.databricks.io.cache.enabled is %s" % spark.conf.get("spark.databricks.io.cache.enabled"))

spark.databricks.io.cache.enabled is false


In [0]:
%pip install dask[complete]

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%pip install pyarrow==10.0.1

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas as pd
import numpy as np
#import databricks.koalas as ks
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

print('pandas version: %s' % pd.__version__)

print('numpy version: %s' % np.__version__)

#print('koalas version: %s' % ks.__version__)

import dask
print('dask version: %s' % dask.__version__)

import pyarrow
print('pyarrow version: %s' % pyarrow.__version__)

import pyspark
print('pyspark version: %s' % pyspark.__version__)


import time

def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")
    return benchmarks['duration'][-1]

def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 10.0.1. Please consider upgrading.
  warnings.warn(
/databricks/python/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


pandas version: 2.2.2
numpy version: 1.26.4
dask version: 2024.5.1
pyarrow version: 10.0.1
pyspark version: 3.3.2.dev0


In [0]:
import pandas as pd

file_path = "/FileStore/tables/yellow_tripdata_2023_01.parquet"
#pandas_data = pd.read_parquet(file_path, engine="pyarrow")
pandas_data = spark.read.format('parquet').options(header='true').load(file_path).toPandas()

In [0]:
pandas_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [0]:
client = Client()

#dask_data = dd.read_parquet("FileStore/tables/teste/yellow_tripdata_2023_02.parquet",index='VendorID')
dask_data = dd.from_pandas(pandas_data, npartitions=3)

dask_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}

In [0]:
import pandas as pd

# Path to your Parquet file
file_path = "dbfs:/FileStore/tables/yellow_tripdata_2023_01.parquet"

# Read the first few rows of the Parquet file
#df_sample = pd.read_parquet(file_path, engine='pyarrow')
df_sample = pandas_data

# Print the column names
print(df_sample.columns)


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


In [0]:
dask_data.head()

/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 142.34 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [0]:
def read_file_parquet(df=None):
    return dd.read_parquet("/FileStore/tables/yellow_tripdata_2023_01.parquet")
  
def count(df=None):
    return len(df)

def count_index_length(df=None):
    return len(df.index)

def mean(df):
    return df.fare_amount.mean().compute()

def standard_deviation(df):
    return df.fare_amount.std().compute()

def mean_of_sum(df):
    return (df.fare_amount + df.tip_amount).mean().compute()

def sum_columns(df):
    return (df.fare_amount + df.tip_amount).compute()

def mean_of_product(df):
    return (df.fare_amount * df.tip_amount).mean().compute()

def product_columns(df):
    return (df.fare_amount * df.tip_amount).compute()
  
def value_counts(df):
    return df.fare_amount.value_counts().compute()
  
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df.start_lon
    phi_1 = df.start_lat
    theta_2 = df.end_lon
    phi_2 = df.end_lat
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.mean().compute()
  
def complicated_arithmetic_operation(df):
    theta_1 = df.start_lon
    phi_1 = df.start_lat
    theta_2 = df.end_lon
    phi_2 = df.end_lat
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.compute()
  
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg(
      {
        'fare_amount': ['mean', 'std'], 
        'tip_amount': ['mean', 'std']
      }
    ).compute()
other = groupby_statistics(dask_data)
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

def join_count(df, other):
    return len(dd.merge(df, other, left_index=True, right_index=True))

def join_data(df, other):
    return dd.merge(df, other, left_index=True, right_index=True).compute()

/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 70.20 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [0]:
#benchmark(read_file_parquet, df=None, benchmarks=dask_benchmarks, name='read file')
benchmark(count, df=dask_data, benchmarks=dask_benchmarks, name='count')
benchmark(count_index_length, df=dask_data, benchmarks=dask_benchmarks, name='count index length')
benchmark(mean, df=dask_data, benchmarks=dask_benchmarks, name='mean')
benchmark(standard_deviation, df=dask_data, benchmarks=dask_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=dask_data, benchmarks=dask_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=dask_data, benchmarks=dask_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=dask_data, benchmarks=dask_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=dask_data, benchmarks=dask_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=dask_data, benchmarks=dask_benchmarks, name='value counts')
# No column for this
# benchmark(mean_of_complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='mean of complex arithmetic ops')
# benchmark(complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='complex arithmetic ops')
benchmark(groupby_statistics, df=dask_data, benchmarks=dask_benchmarks, name='groupby statistics')
benchmark(join_count, dask_data, benchmarks=dask_benchmarks, name='join count', other=other)
benchmark(join_data, dask_data, benchmarks=dask_benchmarks, name='join', other=other) # cant join

count took: 0.03299570083618164 seconds


/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 426.95 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


count index length took: 9.017538070678711 seconds


/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 23.40 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


mean took: 0.4450490474700928 seconds


...

multiplication of columns took: 0.5451595783233643 seconds


/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 23.40 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-05-31 18:31:02,793 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 5d5b6923b743245d00a4e483c6d830a3 initialized by task ('shuffle-transfer-5d5b6923b743245d00a4e483c6d830a3', 1) executed on worker tcp://127.0.0.1:41041
2024-05-31 18:31:04,618 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 5d5b6923b743245d00a4e483c6d830a3 deactivated due to stimulus 'task-finished-1717180264.6163077'
/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 70.20 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(

value counts took: 2.243044376373291 seconds
groupby statistics took: 0.8120646476745605 seconds
join count took: 0.17324423789978027 seconds


/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 435.52 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


join took: 6.59918212890625 seconds
Out[8]: 6.59918212890625

Operations with filtering

In [0]:
expr_filter = (dask_data.tip_amount >= 1) & (dask_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]
  
dask_filtered = filter_data(dask_data)

In [0]:
benchmark(count, dask_filtered, benchmarks=dask_benchmarks, name='filtered count')
benchmark(count_index_length, dask_filtered, benchmarks=dask_benchmarks, name='filtered count index length')
benchmark(mean, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean')
benchmark(standard_deviation, dask_filtered, benchmarks=dask_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, dask_filtered, benchmarks=dask_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, dask_filtered, benchmarks=dask_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered multiplication of columns')
#benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean of complex arithmetic ops')
#benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, dask_filtered, benchmarks=dask_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, dask_filtered, benchmarks=dask_benchmarks, name='filtered groupby statistics')

other = groupby_statistics(dask_filtered)
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_count, dask_filtered, benchmarks=dask_benchmarks, name='filtered join count', other=other)
benchmark(join_data, dask_filtered, benchmarks=dask_benchmarks, name='filtered join', other=other)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 458.92 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


filtered count took: 5.8905229568481445 seconds


/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 458.92 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


filtered count index length took: 6.295364856719971 seconds


/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 46.80 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


...

filtered value counts took: 0.6828720569610596 seconds
filtered groupby statistics took: 0.7611393928527832 seconds


/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 70.20 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 23.40 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


filtered join count took: 0.34832096099853516 seconds


/local_disk0/.ephemeral_nfs/envs/pythonEnv-be5df5f9-6882-4d40-bdb5-d8082643176a/lib/python3.9/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 435.52 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


filtered join took: 5.022407293319702 seconds
Out[10]: 5.022407293319702

In [0]:
dask_res_temp = get_results(dask_benchmarks).set_index('task')
dask_res_temp

,duration
task,
count,0.032996
count index length,9.017538
mean,0.445049
standard deviation,0.454791
mean of columns addition,0.455826
addition of columns,0.418289
mean of columns multiplication,0.428634
multiplication of columns,0.545160
value counts,2.243044
